# 1. Importing Libraries

In [2]:
#importing libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy


In [3]:
# Importing Dataset

In [4]:
path = r'C:\Users\Janvi\Documents\09-2024 Instacart Basket Analysis'

In [5]:
ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', '9_viz.pkl'))

# 2. Addressing the PII data

In [7]:
#dropping PII columns
ords_prods_merge = ords_prods_merge.drop(['firstname', 'surname'], axis=1)

In [8]:
ords_prods_merge.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,eval_set,order_number,order_dow,...,spending_flag,median_days_between_orders,order_frequencey,Gender,state,Age,date_joined,n_dependants,family_status,income
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,prior,28,6,...,High spender,6.5,Frequent customer,Male,Minnesota,81,8/1/2019,1,married,49620
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,prior,30,6,...,High spender,6.5,Frequent customer,Male,Minnesota,81,8/1/2019,1,married,49620
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,prior,2,0,...,Low spender,7.0,Frequent customer,Female,Vermont,66,6/16/2018,2,married,158302
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,prior,1,3,...,Low spender,9.0,Frequent customer,Female,Wisconsin,40,2/9/2020,3,married,31308
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,prior,3,4,...,Low spender,9.0,Frequent customer,Female,Wisconsin,40,2/9/2020,3,married,31308


# 3. comparing customer's behavior in different geographic areas

In [10]:
# creating an array for the state
region = []
# creating a for loop for regions
for state in ords_prods_merge["state"]:
    if state in ['Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 
                 'Rhode Island', 'Connecticut', 'New York', 'Pennsylvania', 
                 'New Jersey']:
        region.append("Northeast")
    elif state in ['Wisconsin', 'Michigan', 'Illinois', 'Indiana', 'Ohio', 
                   'North Dakota', 'South Dakota', 'Nebraska', 'Kansas', 
                   'Minnesota', 'Iowa', 'Missouri']:
        region.append("Midwest")
    elif state in ['Delaware', 'Maryland', 'District of Columbia', 'Virginia', 
                   'West Virginia', 'North Carolina', 'South Carolina', 'Georgia', 
                   'Florida', 'Kentucky', 'Tennessee', 'Mississippi', 'Alabama', 
                   'Oklahoma', 'Texas', 'Arkansas', 'Louisiana']:
        region.append("South")
    elif state in ['Idaho', 'Montana', 'Wyoming', 'Nevada', 'Utah', 'Colorado', 
                   'Arizona', 'New Mexico', 'Alaska', 'Washington', 'Oregon', 
                   'California', 'Hawaii']:
        region.append("West")
    else:
        region.append("Unknown")


In [11]:
# adding the new region list as a column to the df
ords_prods_merge['region'] = region

### checking customer's spending habits in different region

In [13]:
# Step 1: Distinct on user_id level
df_distinct_user = ords_prods_merge.drop_duplicates(subset='user_id')

# Crosstab for user_id level (Region vs Spending Flag)
crosstab_user = pd.crosstab(df_distinct_user['region'], df_distinct_user['spending_flag'], dropna=False)

# Display the crosstabs
print("\nCrosstab for distinct order_id level:")
print(crosstab_user)


Crosstab for distinct order_id level:
spending_flag  High spender  Low spender
region                                  
Midwest                8430        27089
Northeast              6354        20310
South                 11983        38646
West                   9290        29223


# 4. Creating flag for non-active customers

In [15]:
# Creating the column holding maximum orders per user

ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

C:\Users\Janvi\AppData\Local\Temp\ipykernel_4212\1627641474.py:3: FutureWarning: The provided callable <function max at 0x0000023508607C40> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)


In [16]:
# Creating an exclusion flag for low-activity users who have less than 5 orders

ords_prods_merge.loc[ords_prods_merge['max_order'] < 5, 'exclusion_flag'] = 'Low-activity customer'

In [17]:
# Creating an exclusion flag for low-activity users who have equal and more than 5 orders

ords_prods_merge.loc[ords_prods_merge['max_order'] >= 5, 'exclusion_flag'] = 'Regular-activity customer'

In [18]:
ords_prods_merge['exclusion_flag'].value_counts()

exclusion_flag
Regular-activity customer    921270
Low-activity customer         78730
Name: count, dtype: int64

In [19]:
#cross referencing the results
df = ords_prods_merge[ords_prods_merge['exclusion_flag'] != 'Low-activity customer']

In [20]:
df['exclusion_flag'].value_counts()

exclusion_flag
Regular-activity customer    921270
Name: count, dtype: int64

# 5. Create a customer profiling variable based on age, income and certain goods

In [37]:
df['Age'].describe()

count    921270.000000
mean         49.484424
std          18.508229
min          18.000000
25%          33.000000
50%          49.000000
75%          65.000000
max          81.000000
Name: Age, dtype: float64

In [ ]:
# Young Adults: Ages 18 to 34 (inclusive)
df.loc[(df['Age'] >= 18) & (df['Age'] <= 34), 'age_group'] = 'Young adults'

In [ ]:
# Adults: Ages 35 to 64 (inclusive)
df.loc[(df['Age'] >= 35) & (df['Age'] <= 64), 'age_group'] = 'Adults'

In [ ]:
# Elderly Persons: Ages 65 and above
df.loc[(df['Age'] >= 65), 'age_group'] = 'Elderly Adults'

In [39]:
df['income'].describe()

count    921270.000000
mean      99989.377622
std       44934.372047
min       25911.000000
25%       66981.000000
50%       96753.000000
75%      128375.000000
max      593901.000000
Name: income, dtype: float64

# 6. Creating visualization from the results

In [ ]:
# Count unique user_ids for each age_group
age_group_counts = df.groupby('age_group')['user_id'].nunique().sort_index()

# Plot the bar graph with adjusted figure size
plt.figure(figsize=(10, 8))  # Adjust the figure size to make it taller
bar_1 = age_group_counts.plot.bar()
bar_1.set_title('Number of Customers by Age Group')

#### Insight:

Adults (35-64 years old) are the biggest group of customers in our database.